<a href="https://colab.research.google.com/github/englissi/englissi/blob/main/Listening_Cloze_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install gradio
!pip install gTTS
!pip install gtts pydub

In [ ]:
import gradio as gr
import requests
from gtts import gTTS
from pydub import AudioSegment
from pydub.generators import Sine
import os

# Cloze questions
cloze_questions = [
    {
        "question": "In the small mountain village of Echo Ridge, adventure [BLANK] a part of everyday life.",
        "answer": "was",
        "hint": "be"
    },
    {
        "question": "Among these children [BLANK] a bright-eyed eighth grader named Alex.",
        "answer": "was",
        "hint": "be"
    },
    {
        "question": "Alex [BLANK] for his daring spirit and his love for exploring the rugged landscapes around Echo Ridge.",
        "answer": "was known",
        "hint": "know"
    },
    {
        "question": "He [BLANK] a particular fascination with the old maps and tales of hidden treasures that had been lost in the mountains centuries ago.",
        "answer": "had",
        "hint": "have"
    },
    {
        "question": "One day, while exploring the local library, Alex [BLANK] upon an ancient map tucked inside a forgotten book on village lore.",
        "answer": "stumbled",
        "hint": "stumble"
    },
    {
        "question": "The map [BLANK] at the location of a lost treasure, hidden deep within a cave known as Whispering Hollow.",
        "answer": "hinted",
        "hint": "hint"
    },
    {
        "question": "Excited by the prospect of a real adventure, Alex [BLANK] to seek out the treasure.",
        "answer": "decided",
        "hint": "decide"
    },
    {
        "question": "Knowing the journey would be risky, he [BLANK] the help of his best friends, Mia and Sam.",
        "answer": "enlisted",
        "hint": "enlist"
    },
    {
        "question": "Together, they [BLANK] for the expedition, gathering supplies and studying the map extensively.",
        "answer": "prepared",
        "hint": "prepare"
    },
    {
        "question": "They [BLANK] their route, took note of landmarks, and readied themselves for any challenges they might face.",
        "answer": "planned",
        "hint": "plan"
    },
    {
        "question": "Their journey [BLANK] at dawn.",
        "answer": "began",
        "hint": "begin"
    },
    {
        "question": "They [BLANK] through dense forests, crossed rushing streams, and climbed steep cliffs.",
        "answer": "trekked",
        "hint": "trek"
    },
    {
        "question": "Along the way, they [BLANK] various wildlife and navigated through tricky terrains, their map guiding them every step of the way.",
        "answer": "encountered",
        "hint": "encounter"
    },
    {
        "question": "After hours of hiking, they finally [BLANK] Whispering Hollow.",
        "answer": "reached",
        "hint": "reach"
    },
    {
        "question": "The cave [BLANK] more magnificent than they had imagined, filled with intricate stalactites and echoes of dripping water.",
        "answer": "was",
        "hint": "be"
    },
    {
        "question": "Using their flashlights, they [BLANK] deeper into the cave, guided by the markings on the map.",
        "answer": "ventured",
        "hint": "venture"
    },
    {
        "question": "As they [BLANK] the heart of the cave, they discovered an ancient chest hidden behind a fallen boulder.",
        "answer": "reached",
        "hint": "reach"
    },
    {
        "question": "With hearts pounding, they [BLANK] the boulder and opened the chest.",
        "answer": "moved",
        "hint": "move"
    },
    {
        "question": "Inside, instead of gold or jewels, they [BLANK] a collection of old artifacts: pottery, coins, and a beautifully carved statuette of an eagle — the Guardian of the Glen.",
        "answer": "found",
        "hint": "find"
    },
    {
        "question": "Realizing the historical significance of their find, they [BLANK] to donate the artifacts to the local museum.",
        "answer": "decided",
        "hint": "decide"
    },
    {
        "question": "The village [BLANK] their discovery, and the children were hailed as heroes.",
        "answer": "celebrated",
        "hint": "celebrate"
    },
    {
        "question": "Their adventure [BLANK] the community together, sparking a renewed interest in the history and legends of Echo Ridge.",
        "answer": "brought",
        "hint": "bring"
    },
    {
        "question": "Alex, Mia, and Sam [BLANK] local legends, known not only for their daring but also for their spirit of discovery and respect for heritage.",
        "answer": "became",
        "hint": "become"
    },
    {
        "question": "They [BLANK] to explore the mountains, each adventure strengthening their friendship and deepening their connection to their village.",
        "answer": "continued",
        "hint": "continue"
    },
    {
        "question": "The legend of the Guardian of the Glen [BLANK] on, not just as a protector but as a symbol of adventure and discovery, inspiring future generations to explore the mysteries of Echo Ridge.",
        "answer": "lived",
        "hint": "live"
    }
]

# Function to create a bell sound
def create_bell_sound(filename="bell.wav"):
    # Generate a bell sound (1000 Hz tone for 200 ms)
    tone = Sine(1000).to_audio_segment(duration=200).apply_gain(-10).fade_in(50).fade_out(50)
    tone.export(filename, format="wav")

# Create the bell sound file
create_bell_sound()

# Function to convert text to speech and add bell sound
def text_to_speech_with_bell(text, filename):
    tts = gTTS(text.replace("[BLANK]", ""))
    tts.save("temp.mp3")

    # Load the generated speech and bell sound
    speech = AudioSegment.from_mp3("temp.mp3")
    bell = AudioSegment.from_wav("bell.wav")

    # Find the position of the blank and insert the bell sound
    blank_position = text.find("[BLANK]")
    part1 = text[:blank_position]
    tts_part1 = gTTS(part1).save("part1.mp3")
    speech_part1 = AudioSegment.from_mp3("part1.mp3")

    combined = speech_part1 + bell + speech[len(speech_part1):]

    # Save the final audio with the bell sound inserted
    combined.export(filename, format="mp3")
    os.remove("temp.mp3")
    os.remove("part1.mp3")
    return filename

# Generate audio files for questions
for i, question in enumerate(cloze_questions):
    audio_filename = f"question_{i+1}.mp3"
    question["audio"] = text_to_speech_with_bell(question["question"], audio_filename)

# Function to handle the cloze quiz
def cloze_quiz(state, answer):
    name, question_index, score, results = state
    question = cloze_questions[question_index]

    correct = answer.strip().lower() == question["answer"].lower()

    if correct:
        score += 1
        results.append(f"Question {question_index + 1}: Correct\n")
    else:
        results.append(f"Question {question_index + 1}: Incorrect, the correct answer is: {question['answer']}\n")

    question_index += 1

    if question_index < len(cloze_questions):
        next_question_audio = cloze_questions[question_index]["audio"]
        next_hint = f"Hint: {cloze_questions[question_index]['hint']} (동사 형태 변형 가능)"
        return (name, question_index, score, results), next_question_audio, next_hint, gr.update(visible=False), gr.update(value="")
    else:
        result_text = f"* Name: {name}\n* Score: {score} out of {len(cloze_questions)}\n" + "\n".join(results)
        return (name, question_index, score, results), None, "", gr.update(visible=True, value=result_text), gr.update(visible=False)

# Function to start the quiz
def start_quiz(name):
    hint = f"Hint: {cloze_questions[0]['hint']} (동사 형태 변형 가능)"
    return (name, 0, 0, []), cloze_questions[0]["audio"], hint, gr.update(visible=False), gr.update(visible=True)

# Create the Gradio interface
with gr.Blocks() as iface:
    gr.Markdown("# Listening Cloze Test (27문항)")
    gr.Markdown("**신호음이 들리는 빈칸에 들어갈 동사의 형태를 써보세요. (문장 전체를 쓰지 않습니다.)**")
    name_input = gr.Textbox(label="Enter your name")
    start_button = gr.Button("Start Quiz")
    question_audio = gr.Audio(interactive=False)
    hint_output = gr.Markdown()
    answer_input = gr.Textbox(label="Your Answer")
    next_button = gr.Button("Next")
    result_output = gr.Textbox(label="Results", interactive=False, visible=False)

    # Initialize the state
    state = gr.State()

    start_button.click(start_quiz, inputs=name_input, outputs=[state, question_audio, hint_output, result_output, answer_input])
    next_button.click(cloze_quiz, inputs=[state, answer_input], outputs=[state, question_audio, hint_output, result_output, answer_input])

    # Add the hyperlink and image
    gr.Markdown("<img src='https://github.com/englissi/englissi/blob/4f8a7cc7c7194132422e05081fd7d9502c3c4c65/Sample/image%20matching.webp' alt='Image' />")
    gr.Markdown("[Click here to visit the matching game](http://englissi-matching.hf.space)")

iface.launch(share=True)
